In [4]:
import pandas as pd
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import plotting
from pypfopt import CLA
from pypfopt import hierarchical_portfolio
import yfinance as yf
import datetime as dt

### Get all the stock tickers of Sensex

In [7]:
ticker = ["TCS.BO","LT.BO","INFY.BO","AXISBANK.BO","HINDUNILVR.BO","HCLTECH.BO","WIPRO.BO","ICICIBANK.BO","BAJAJFINSV.BO","RELIANCE.BO","TECHM.BO","BAJFINANCE.BO","M&M.BO","MARUTI.BO","ULTRACEMCO.BO","POWERGRID.BO","SBIN.BO","BHARTIARTL.BO","TITAN.BO","INDUSINDBK.BO","ITC.BO","HDFCBANK.BO","TATASTEEL.BO","TATAMOTORS.BO","NTPC.BO","ASIANPAINT.BO","KOTAKBANK.BO","JSWSTEEL.BO","SUNPHARMA.BO","NESTLEIND.BO"]
prices=pd.DataFrame()

### Extract data for last 10 years for all the stock tickers

In [10]:
end_time=dt.datetime.now()
years=10
start_time = end_time - dt.timedelta(days=365*years)
for i in ticker:
    prices_data=yf.download(i,start_time,end_time)
    prices[i]=prices_data['Adj Close']
print(prices)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

### Calculate daily returns

In [12]:
daily_return=prices.pct_change().dropna()
print(daily_return)

              TCS.BO     LT.BO  ...  SUNPHARMA.BO  NESTLEIND.BO
Date                            ...                            
2014-01-13  0.038785  0.025209  ...     -0.011891      0.001349
2014-01-14 -0.017783 -0.007805  ...      0.001909     -0.007684
2014-01-15  0.011211  0.027096  ...      0.000083     -0.027480
2014-01-16 -0.001232  0.004005  ...     -0.017229     -0.017015
2014-01-17 -0.057712 -0.001645  ...      0.001517      0.018525
...              ...       ...  ...           ...           ...
2024-01-01  0.004124 -0.001333  ...     -0.000079      0.028947
2024-01-02 -0.007177 -0.023635  ...      0.028971     -0.004913
2024-01-03 -0.024238  0.000567  ...      0.001311     -0.021687
2024-01-04 -0.006650  0.005260  ...      0.011209      0.017855
2024-01-05  0.019294  0.016160  ...     -0.009751     -0.016485

[2441 rows x 30 columns]


C:\Users\manda\AppData\Local\Temp\ipykernel_22884\3321074472.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_return=prices.pct_change().dropna()
